1. Imports & paths

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

train_dir = "/content/brain_dataset/Training"
test_dir  = "/content/brain_dataset/Testing"

IMG_SIZE = 224
BATCH_SIZE = 16   # medical datasets are usually smaller


2. Load dataset

In [7]:
import zipfile
import os

zip_path = '/content/brain_dataset.zip'
extract_path = '/content/'

print(f"Unzipping {zip_path} to {extract_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Successfully extracted {zip_path} to {extract_path}")

# Verify content (optional, but good for confirmation)
print("Contents of extracted folder (first level):")
for item in os.listdir(os.path.join(extract_path, 'brain_dataset')):
    print(f"  - {item}")


Unzipping /content/brain_dataset.zip to /content/...
Successfully extracted /content/brain_dataset.zip to /content/
Contents of extracted folder (first level):
  - Testing
  - Training


In [16]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    "brain_dataset/Training",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode="rgb",
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "brain_dataset/Testing",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode="rgb",
    shuffle=False
)


Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


3. Preprocessing

In [17]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [18]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [19]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input


5. Classification head

In [23]:
num_classes = 4 # Explicitly set to 4 based on dataset output

base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # IMPORTANT

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


6. Compile & train (Phase 1)

In [25]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,427,588 (9.26 MB)

 Trainable params: 167,044 (652.52 KB)

 Non-trainable params: 2,260,544 (8.62 MB)

In [28]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        mode="max",
        patience=7,              # allow some ups & downs
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_accuracy",  # NOT val_loss
        mode="max",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]


In [29]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks
)


Epoch 1/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 62s 687ms/step - accuracy: 0.8568 - loss: 0.4011 - val_accuracy: 0.6726 - val_loss: 1.3444 - learning_rate: 3.0000e-04
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 62s 687ms/step - accuracy: 0.8537 - loss: 0.3745 - val_accuracy: 0.6751 - val_loss: 1.4237 - learning_rate: 3.0000e-04
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 62s 686ms/step - accuracy: 0.8622 - loss: 0.3548 - val_accuracy: 0.6853 - val_loss: 1.4888 - learning_rate: 3.0000e-04
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 62s 686ms/step - accuracy: 0.8735 - loss: 0.3222 - val_accuracy: 0.6802 - val_loss: 1.5128 - learning_rate: 3.0000e-04
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 62s 686ms/step - accuracy: 0.8727 - loss: 0.3218 - val_accuracy: 0.6980 - val_loss: 1.5180 - learning_rate: 3.0000e-04
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 63s 706ms/step - accuracy: 0.8822 - loss: 0.2976 - val_accuracy: 0.6878 - val_loss: 1.5811 - learning_rate: 3.0000e-04
Epoch 7/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 63s 703ms/step - acc

7. Fine-tuning (Phase 2 – where gains happen)

In [31]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False


In [32]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 80s 832ms/step - accuracy: 0.8252 - loss: 0.4827 - val_accuracy: 0.6853 - val_loss: 1.1947 - learning_rate: 1.0000e-05
Epoch 2/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 74s 825ms/step - accuracy: 0.8330 - loss: 0.4265 - val_accuracy: 0.6650 - val_loss: 1.3023 - learning_rate: 1.0000e-05
Epoch 3/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 74s 824ms/step - accuracy: 0.8541 - loss: 0.3786 - val_accuracy: 0.6548 - val_loss: 1.3892 - learning_rate: 1.0000e-05
Epoch 4/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 747ms/step - accuracy: 0.8646 - loss: 0.3596
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
90/90 ━━━━━━━━━━━━━━━━━━━━ 74s 821ms/step - accuracy: 0.8647 - loss: 0.3594 - val_accuracy: 0.6497 - val_loss: 1.4382 - learning_rate: 1.0000e-05
Epoch 5/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 74s 822ms/step - accuracy: 0.8880 - loss: 0.2912 - val_accuracy: 0.6675 - val_loss: 1.4326 - learning_rate: 5.0000e-06
Epoch 6/15
90/90 ━━━━━━━━━━━━━━━━━━━━ 74s 827ms/step - accuracy:

8. Evaluate


In [ ]:
loss, acc = model.evaluate(test_ds)
print(f"Test Accuracy: {acc:.4f}")